In [ ]:
#!pip -q install git+https://github.com/huggingface/transformers # need to install from github
#!pip -q install accelerate>=0.12.0
!pip install transformers
!pip install datasets
!pip install sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
threshold = 0.25

model = SentenceTransformer('all-mpnet-base-v2', device='cuda' if torch.cuda.is_available() else "cpu")
# model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda' if torch.cuda.is_available() else "cpu")
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)
embed_data = torch.load('train_embeddings.pt')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gpt2 = pipeline(model="gpt2-large", max_new_tokens=384, device='cuda' if torch.cuda.is_available() else "cpu", return_full_text=False)
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")

In [ ]:
def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)





def get_similar(element, items=None, embeddings=None, num=None):
    emb_items = None

    if items is None and embeddings is not None:
        emb_items = embeddings["emb_questions"]
        items = embeddings["keys"]
    elif items is not None:
        emb_items = model.encode(items)

    if len(element) == 0 or emb_items is None:
        return []

    emb_element = model.encode(element)

    result = []
    scores = cos_sim(emb_element, emb_items)

    if num is None or num < 2:
        maximus = torch.max(scores, 1)
        m = float(maximus.values[0])
        i = int(maximus.indices[0])
        if m > threshold:
            result = [[round(m, 4), items[i], embeddings["questions"][i], embeddings["queries"][i]]]
        return result
    else:
        scored_texts = []
        for i, score in enumerate(scores[0]):
            scored_texts.append([round(score.item(), 4), items[i], embeddings["questions"][i], embeddings["queries"][i]])
        sorted_scored_texts = sorted(scored_texts, key=lambda x: x[0], reverse=True)
        return sorted_scored_texts[:num]


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def get_key(q):
    t0 = q.get('template_id')
    if t0 is None:
        t0 = "None"
    t = str(q.get("number_of_patterns")) + "-" + t0
    return t


def save_embedding():
    train = raw_datasets.get("train")
    questions = [q["question"]["string"] for q in train]
    queries = [clean(q["query"]["sparql"]) for q in train]
    keys = [get_key(q) for q in train]
    embeddings = {}
    emb_questions = model.encode(questions)
    embeddings["questions"] = questions
    embeddings["emb_questions"] = emb_questions
    embeddings["queries"] = queries
    embeddings["keys"] = keys
    torch.save(embeddings, 'train_embeddings.pt')
    return embeddings


def prepare_queries(n_):
    data = raw_datasets.get("test")
    queries = []
    suggestions = []
    for q in data:
        t = get_key(q)
        question = q["question"]["string"]
        suggestion = get_similar(question,embeddings=embed_data, num=n_)
        suggestions.append([[[x[0], x[1]] for x in suggestion],t])
        # print(suggestion, t)
        # break
        # continue
        if suggestion is None or len(suggestion)==0:
            print("Error with key", t)
            queries.append("translate the following English text '" + question + "' to a sparql query")
        else:
            final_q = ""
            for i_, k in enumerate(suggestion):
                final_q += "\n input (English text): " + k[2]
                final_q += "\n output (Sparql query): " + k[3]

            # works better with gpt
            # final_q += "\n with this example what is the sparql query for:  " + question

            # works better with dolly
            final_q += "\n input (English text): " + question
            final_q += "\n output (Sparql query): "
            queries.append(final_q)
    return queries, suggestions


def main(shots=1,attempts=10,batch=50):
    # save_embedding()

    query_list, suggestions = prepare_queries(shots)

    print(len(query_list))
    # print(query_list, suggestions)
    # quit()

    n = batch
    q_list = query_list  #list(divide_chunks(query_list, n))
    sparql = [clean(x["query"]["sparql"]) for x in raw_datasets.get("test")]

    gs = []
    lens =[]

    for i, question in enumerate(q_list):
      print(i, question)
      res_ = tokenizer.encode(question)
      len_ = len(res_)
      lens.append(len_)
      print(len_)
      if len_ > 620:
        print(type(res_))
        question = tokenizer.decode(res_[-620:])
      res = gpt2(question)
      gs.append(res[0]["generated_text"])
      print(res[0]["generated_text"])
      result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens, "suggestions": suggestions}
      save_json("nlp_gpt2_"+str(shots)+"_shot_results_tok.json", result)


In [ ]:
main(shots=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


513
0 
 input (English text): Can you provide the highest benchmark result, including the metric and score, for the Story Cloze Test dataset?
 output (Sparql query): SELECT DISTINCT ?metric ?metric_lbl (MAX( ?value) AS ?score) WHERE { { SELECT ?metric ?metric_lbl ?value WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Story Cloze Test") ?benchmark orkgp:HAS_DATASET ?dataset; orkgp:HAS_EVALUATION ?eval. ?eval orkgp:HAS_VALUE ?value. OPTIONAL { ?eval orkgp:HAS_METRIC ?metric. ?metric rdfs:label ?metric_lbl. } ?cont orkgp:HAS_BENCHMARK ?benchmark. OPTIONAL { ?cont orkgp:HAS_MODEL ?model. ?model rdfs:label ?model_lbl. } } ORDER BY DESC( ?value) } } GROUP BY ?metric ?metric_lbl
 input (English text): Can you list the models that have been evaluated on the Story Cloze Test dataset?
 output (Sparql query): SELECT DISTINCT ?model ?model_lbl WHERE { ?dataset a orkgc:Dataset; rdfs:label ?dataset_lbl. FILTER (str( ?dataset_lbl) = "Story Cloze Test") ?benchm

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



If all the filters or conditions have been satisfied, the output will be {% name %} {% e.g. story_cloze_test_acc = {% e.g. paper orkgc:Metric }, % attrs:dataset, % attrs:metric, and % attrs:model %}. 
input (English text): Describe the methods used to find the model that has achieved the highest Accuracy score on the WSC benchmark dataset?
output (Sparql query): SELECT?name,?accuracy,?model,?dataset as % model_values FROM ( SELECT accuracy, model as model_value FROM e.model AS e WHERE e.model_type ='story_cloze_test')?benchmark ;  orkgp:HAS_DATASET?dataset; orkgp:HAS_EVALUATION?eval.?paper orkgp:HAS_INITIALIZE?dataset. ORDER BY desc(?accuracy) LIMIT 1; orkgp:HAS_METRIC?metric; orkgp:HAS_BENCHMARK?benchmark. OPTIONAL {?paper orkgp:HAS_MODEL?model.?model rdfs:label?model_lbl. }?con orkgp:HAS_MODEL?model?model rdfs:label?model_lbl. ORIENT {?metric orkgp:HAS_DATASET?dataset; rdfs:label?model_lbl. }?orderby orkgp:HAS_MODEL?model. ORIENT {?metric orkgp:HAS_DATASET?dataset
1 
 input (English

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

 select name, metric, score... as score from qbert_tbl;
Output:
Name =?
Measuring the performance of our models using benchmark scores is more about seeing the difference in performance between two (or more) models versus comparing their different parameters. The advantage this approach brings me is that, in comparison to a more traditional clustering model, I can use RDF to visualize changes in the model. In this case, I can see how the model changed the most as compared to the others (or the same).
1. We can check which model has accumulated the most score
output (Sparql query): SELECT DISTINCT?model?model_lbl FROM ( SELECT?dataset metric* as metric,?name performance? as performance,?model performance? as model,?name metric* as metric_lbl from (SELECT?dataset metric* as metric,?name performance? as performance,?model performance? as model,?name metric* as metric_lbl FROM qbert_tbl AS performance L WHERE metric[metric] IS NULL ) INNER JOIN metrics_lbl_lbl AS performance_lbl LEFT OUTER

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 <scr> from {?col1} to {?col*}.score AS{?score_lbl:label?model}} select {?table_lbl:{?col1}.score.from_lbl.data_lbl.score_metrics from {?col1}.table to {?col-1}.table where {?col1.score = 'One Billion Word'} or [{?tablename.score = 'SQR'}.score.from_lbl. FILTER {?str(?column1.value) ='One Billion Word';}}] order by {?column1.score} desc or {?column1.score} desc or {?column1} desc
 input (English text): What is the top metric when the metric scores are combined?
 output (Sparql query):  SELECT TOP {?metric?metric_lbl} score. FROM {?tablename.topScore. score} to {?col1}.topScore where {?col1.score = 'SQR'} or [{?tablename.topScore.score = 'SQR';}}] ORDER BY {?metric?metric_lbl} desc?
The results look like this:
Sets of performance metrics for the benchmarks:
input (English text): What are the metrics of evaluation of the Words in Context dataset?
output (Sparql query):  <scr> from {?col1} to {?col*}.score AS{?score_lbl:label?model} SELECT TOP {?col1} score. FROM {?col1}.table to {?col-1}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 SELECT DISTINCT?dataset?dataset_lbl WHERE {?problem a orkgc:Problem; rdfs:label?problem_lbl. FILTER (str(?problem_lbl) = "Common Sense Reasoning")?dataset a orkgc:Dataset; rdfs:label?dataset_lbl.?benchmark orkgp:HAS_DATASET?dataset.?cont orkgp:HAS_BENCHMARK?benchmark; orkgp:P32?problem. } 
input (English text): Name the datasets that have been used for the benchmarking in the Knowledge Graph Construction research problem?
output (Sparql query):  SELECT DISTINCT?dataset?dataset_lbl WHERE {?problem a orkgc:Problem; rdfs:label?problem_lbl. FILTER (str(?problem_lbl) = "Knowledge Graph Construction")?dataset a orkgc:Dataset; rdfs:label?dataset_lbl.?benchmark orkgp:HAS_DATASET?dataset.?cont orkgp:HAS_BENCHMARK?benchmark; orkgp:P32?problem. } 
input (English text): Name the datasets that have been used for the benchmarking in the Common Sense Reasoning research problem?
output (Sparql query):  SELECT DISTINCT?dataset?dataset_lbl WHERE {?problem a orkgc:Problem; rdfs:label?problem_lbl. FILTER